# Colab: 两城市强制 Mamba 验证

本 Notebook 按顺序逐格运行即可：
1. 挂载 Google Drive
2. 克隆仓库
3. 复制 `20152024.xlsx` 到项目根目录
4. 安装 CUDA PyTorch + mamba-ssm
5. 强制 Mamba 两城市训练
6. 生成 loss/pred/MAE/RMSE 可视化并回传到 Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 这里改成你的仓库地址
REPO_URL = 'https://github.com/<your-org-or-user>/<your-repo>.git'

# 你的数据在 Drive 的路径
DRIVE_XLSX = '/content/drive/MyDrive/rainfall/20152024.xlsx'

# 结果回传目录
DRIVE_OUT_DIR = '/content/drive/MyDrive/rainfall_outputs'

print('REPO_URL =', REPO_URL)
print('DRIVE_XLSX =', DRIVE_XLSX)
print('DRIVE_OUT_DIR =', DRIVE_OUT_DIR)

In [ ]:
!nvidia-smi

In [ ]:
%cd /content
!rm -rf rainfall
!git clone "$REPO_URL" rainfall
%cd /content/rainfall
!ls -la

In [ ]:
!cp "$DRIVE_XLSX" /content/rainfall/20152024.xlsx
!ls -lh /content/rainfall/20152024.xlsx

In [ ]:
!python -m pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt
!pip install mamba-ssm

In [ ]:
import importlib.util
import torch
print('cuda_available =', torch.cuda.is_available())
print('mamba_installed =', importlib.util.find_spec('mamba_ssm') is not None)
assert torch.cuda.is_available(), 'GPU 不可用：请确认 Colab Runtime 已切换到 GPU'
assert importlib.util.find_spec('mamba_ssm') is not None, 'mamba_ssm 未安装成功'

In [ ]:
%env PYTHONPATH=.
!python -u -m scripts.train --config configs/gz_rain_quick_2cities_mamba.yaml --device cuda 2>&1 | tee runs/train_2cities_mamba.log

In [ ]:
!python scripts/plot_results.py --exp_dir runs/gz_rain_da_deformmamba_2cities_mamba --horizon 1 --tail_points 300
!ls -la runs/gz_rain_da_deformmamba_2cities_mamba/plots

In [ ]:
!mkdir -p "$DRIVE_OUT_DIR"
!cp -r runs/gz_rain_da_deformmamba_2cities_mamba "$DRIVE_OUT_DIR/"
!tar -czvf /content/gz_rain_da_deformmamba_2cities_mamba.tgz -C runs gz_rain_da_deformmamba_2cities_mamba
print('Saved folder ->', DRIVE_OUT_DIR + '/gz_rain_da_deformmamba_2cities_mamba')
print('Saved tar -> /content/gz_rain_da_deformmamba_2cities_mamba.tgz')